<a href="https://colab.research.google.com/github/a6iyyu/MachineLearning_3F_23/blob/main/JS10/src/lab-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🚐 Lab 3**

Klasifikasi Naive Bayes dengan Data Multimoda.

### Langkah 1 - _Load_ Data

Pada tahap ini kita akan _loading_ data ke dalam data frame dan melakukan inspeksi sederhana untuk memastikan apakah kita perlu proses pra pengolahan data sebelum melakukan ekstraksi fitur dan permodelan.

In [66]:
from google.colab import files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import io
import numpy as np
import pandas as pd

In [67]:
uploaded = files.upload()
filename = 'spam.csv'

# Periksa apakah file yang benar diupload
if filename in uploaded:
    try:
        # Baca konten file yang sudah diupload
        df = pd.read_csv(
            io.BytesIO(uploaded[filename]),
            encoding='latin-1'
        )

        print(f"\nFile '{filename}' berhasil dibaca. Ini 5 baris pertamanya:")
        print(df.head())

    except pd.errors.EmptyDataError:
        print(f"Error: File '{filename}' ditemukan, tapi isinya kosong.")

    except Exception as e:
        print(f"Terjadi error saat mem-parsing file: {e}")
else:
    # Ini menangani kasus jika user meng-cancel atau upload file yang salah
    print(f"\nError: File '{filename}' tidak ditemukan di antara file yang diupload.")

Saving spam.csv to spam.csv

File 'spam.csv' berhasil dibaca. Ini 5 baris pertamanya:
     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


### Langkah 2 - Preprocessing

Beberapa hal yang akan dilakukan pada tahap ini yaitu,

- Drop kolom yang tidak digunakan

- Ubah nama kolom v1 (label) dan v2 (teks sms)

- Inspeksi Data

- Encode label

- Memisahkan fitur dengan label

### Langkah 2a - Drop Kolom

In [68]:
# Drop 3 kolom terakhir dengan fungsi iloc
df = df.drop(df.iloc[:,2:], axis=1)

# Cek data
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### Langkah 2b - Inspeksi Data

In [69]:
df.columns = ['Labels', 'SMS']

# Cek Jumlah Data Per Kelas (Sekarang 'Labels' sudah ada)
print("Jumlah Data Per Kelas:")
print(df['Labels'].value_counts())
print('\n')

# Cek Kelengkapan Data
print(df.info())
print('\n')

# Cek Statistik Deskriptif
print(df.describe())

Jumlah Data Per Kelas:
Labels
ham     4825
spam     747
Name: count, dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Labels  5572 non-null   object
 1   SMS     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None


       Labels                     SMS
count    5572                    5572
unique      2                    5169
top       ham  Sorry, I'll call later
freq     4825                      30


### Langkah 2c - Encoding Label

In [70]:
# Data untuk label
new_labels = {
    'spam': 1,
    'ham': 0
}

# Encode label
df['Labels'] = df['Labels'].map(new_labels)

# Cek data
df.head()

,Labels,SMS
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


### Langkah 2d - Pisahkan Fitur dengan Label

In [71]:
df.dropna(inplace=True)
X = df['SMS'].values
y = df['Labels'].values

### Langkah 3 - Ekstraksi Fitur

Ekstraksi fitur untuk setiap SMS akan menggunakan konsep Bag of Words. Kita dapat menggunakan fungsi `CountVectorizer` dari scikit-learn. Akan tetapi untuk mencegah **leaking information** kita akan melakukan split data terlebih dahulu, baru melakukan transformasi terhadap data training dan testing.

In [72]:
# Split data training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

# Inisiasi CountVectorizer
bow = CountVectorizer()

# Fitting dan transform X_train dengan CountVectorizer
X_train = bow.fit_transform(X_train)

# Transform X_test
# Mengapa hanya transform? Alasan yang sama dengan kasus pada percobaan ke-3
# Kita tidak menginginkan model mengetahui paramter yang digunakan oleh CountVectorizer untuk fitting data X_train
# Sehingga, data testing dapat tetap menjadi data yang asing bagi model nantinya
X_test = bow.transform(X_test)

Cek fitur dari proses `CountVectorizer`.

In [73]:
print(len(bow.get_feature_names_out()))
print(f'Dimensi data: {X_train.shape}')

7727
Dimensi data: (4457, 7727)


### Langkah 4 - Training dan Evaluasi Model

Kita akan menggunakan algoritma Multinomial Naive Bayes. Fungsi `MultinomialNB` dari scikit-learn dapat digunakan pada kasus ini.

In [74]:
# Inisiasi MultinomialNB
mnb = MultinomialNB()

# Fit model
mnb.fit(X_train, y_train)

# Prediksi dengan data training
y_pred_train = mnb.predict(X_train)

# Evaluasi akurasi data training
acc_train = accuracy_score(y_train, y_pred_train)

# Prediksi dengan data training
y_pred_test = mnb.predict(X_test)

# Evaluasi akurasi data training
acc_test = accuracy_score(y_test, y_pred_test)

# Print hasil evaluasi
print(f'Hasil akurasi data train: {acc_train}')
print(f'Hasil akurasi data test: {acc_test}')

Hasil akurasi data train: 0.9946152120260264
Hasil akurasi data test: 0.9775784753363229
